In [21]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame

In [22]:
spark = SparkSession.builder.getOrCreate()

In [23]:
time_zone = "Europe/Rome" # America/New_York
time_zone_utc = "UTC"

spark.conf.set('spark.sql.session.timeZone', time_zone)

In [31]:
data_df = spark.createDataFrame([
    (1, "2023-03-25 23:00"),
    (1, "2023-03-26 00:00"),
    (1, "2023-03-26 01:00"),
    (1, "2023-03-26 01:59"),
    # (1, "2023-03-26 02:00"),
    (1, "2023-03-26 03:00"),
    (1, "2023-03-26 04:00"),
    (1, "2023-03-26 10:00"),
], schema=["id", "datetime"])

data_df = data_df.withColumn("datetime", F.to_timestamp(F.col("datetime"), "yyyy-MM-dd HH:mm"))

In [33]:
windows_size = "1 hour"

data_df.show(truncate=False)

+---+-------------------+
|id |datetime           |
+---+-------------------+
|1  |2023-03-25 23:00:00|
|1  |2023-03-26 00:00:00|
|1  |2023-03-26 01:00:00|
|1  |2023-03-26 01:59:00|
|1  |2023-03-26 03:00:00|
|1  |2023-03-26 03:00:00|
|1  |2023-03-26 04:00:00|
|1  |2023-03-26 10:00:00|
+---+-------------------+



23/07/14 19:08:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1003300 ms exceeds timeout 120000 ms
23/07/14 19:08:28 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/14 19:25:57 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

In [26]:
data1_df = data_df.withColumn("utc_datetime", F.to_utc_timestamp(F.col("datetime"), time_zone))
data1_df.show()

+---+-------------------+-------------------+
| id|           datetime|       utc_datetime|
+---+-------------------+-------------------+
|  1|2023-03-25 23:00:00|2023-03-25 22:00:00|
|  1|2023-03-26 00:00:00|2023-03-25 23:00:00|
|  1|2023-03-26 01:00:00|2023-03-26 00:00:00|
|  1|2023-03-26 01:59:00|2023-03-26 00:59:00|
|  1|2023-03-26 03:00:00|2023-03-26 01:00:00|
|  1|2023-03-26 04:00:00|2023-03-26 03:00:00|
|  1|2023-03-26 10:00:00|2023-03-26 08:00:00|
+---+-------------------+-------------------+



ERRORE: Notare in cella precedente come le 4 datetime dovrebbero essere le 2 utc_datetime

In [27]:
# spark.conf.set('spark.sql.session.timeZone', "UTC")

data1_df = data1_df.withColumn("utc_window_utc_date", F.window(timeColumn=F.col("utc_datetime"), windowDuration=windows_size))
data1_df.show(truncate=False)

+---+-------------------+-------------------+------------------------------------------+
|id |datetime           |utc_datetime       |utc_window_utc_date                       |
+---+-------------------+-------------------+------------------------------------------+
|1  |2023-03-25 23:00:00|2023-03-25 22:00:00|{2023-03-25 22:00:00, 2023-03-25 23:00:00}|
|1  |2023-03-26 00:00:00|2023-03-25 23:00:00|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|
|1  |2023-03-26 01:00:00|2023-03-26 00:00:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|
|1  |2023-03-26 01:59:00|2023-03-26 00:59:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|
|1  |2023-03-26 03:00:00|2023-03-26 01:00:00|{2023-03-26 01:00:00, 2023-03-26 03:00:00}|
|1  |2023-03-26 04:00:00|2023-03-26 03:00:00|{2023-03-26 03:00:00, 2023-03-26 04:00:00}|
|1  |2023-03-26 10:00:00|2023-03-26 08:00:00|{2023-03-26 08:00:00, 2023-03-26 09:00:00}|
+---+-------------------+-------------------+------------------------------------------+



In [30]:
data1_df = data1_df.withColumn("window_date", F.window(timeColumn=F.col("datetime"), windowDuration=windows_size))
# data1_df = data1_df.withColumn("window_utc_date", F.window(timeColumn=F.col("datetime_utc"), windowDuration=windows_size))
data1_df.show(truncate=False)

+---+-------------------+-------------------+------------------------------------------+------------------------------------------+
|id |datetime           |utc_datetime       |utc_window_utc_date                       |window_date                               |
+---+-------------------+-------------------+------------------------------------------+------------------------------------------+
|1  |2023-03-25 23:00:00|2023-03-25 22:00:00|{2023-03-25 22:00:00, 2023-03-25 23:00:00}|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|
|1  |2023-03-26 00:00:00|2023-03-25 23:00:00|{2023-03-25 23:00:00, 2023-03-26 00:00:00}|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|
|1  |2023-03-26 01:00:00|2023-03-26 00:00:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|{2023-03-26 01:00:00, 2023-03-26 03:00:00}|
|1  |2023-03-26 01:59:00|2023-03-26 00:59:00|{2023-03-26 00:00:00, 2023-03-26 01:00:00}|{2023-03-26 01:00:00, 2023-03-26 03:00:00}|
|1  |2023-03-26 03:00:00|2023-03-26 01:00:00|{2023-03-26 01:00:00, 2023-03-2

In [29]:
# spark.conf.set('spark.sql.session.timeZone', "UTC")

# data1_df = data1_df.withColumn("utc_window_date", F.window(timeColumn=F.col("datetime"), windowDuration=windows_size))
# data1_df = data1_df.withColumn("utc_window_utc_date", F.window(timeColumn=F.col("datetime_utc"), windowDuration=windows_size))

# spark.conf.set('spark.sql.session.timeZone', time_zone)

# data1_df.show(truncate=False)

## Appunti

Ho capito che bisogna fare il bucketing impostando la time zone a UTC altrimenti i bucket vengono fuori non uniformi